<a href="https://colab.research.google.com/github/ramirezzzluisss/IS4487/blob/main/Week13Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS 4487 Module 5 Script

## Objectives

What do you need to know for this module's project?

1. Fit a tree model using all the available predictors. 
2. Create a confusion matrix  and identify numbers of TP, FP, TN and FN.
3. Estimate profit (benefits - costs) using a defined cost-benefit matrix and the above confusion matrix.
4. Estimate profit for a strategy that consists in calling everyone.
5. Estimate profit for a class decision threshold of .3 
6.  Use the model to predict on a new dataset (without the target), then use these predictions to identify those who should be called--a contact list.
7.  Make a recommendation to the Director of Sales based on your analytic work.

**NOTE:** For the module 5 project you will predict answering for prospective customers (who have not yet been called). Consequently, the assignment requires an additional dataset, customer_data.csv, that you will need to download and move into your project folder.  Here is code to load that data into memory after you have it saved in your project folder (your working directory). This code is included in the template script.

We will again use the MegaTelCo data for this demonstration.

##Load Libraries

In this class we will be using 
- Pandas
- Scikitlearn
- Matplotlib


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import statsmodels.api as sm
import datetime as dt

from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics  #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix
from sklearn import linear_model


## Getting data into Pandas

In this case we will load data from the statsmodels.org library

See the Canvas assignments and lectures for a description of the Megatelco data

**Note:** you will need to enter a code supplied by Google in the next step. 


In [6]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

df = pd.read_csv (r'/content/gdrive/MyDrive/Colab Notebooks/retail_dataset.csv')


Mounted at /content/gdrive


In [7]:
#look at the top rows
df.head(10) 

,customer_id,customer_name,address_line_1,city,state_province,postal_code,order_id,order_date,customer_id.1,order_id.1,product_id,quantity,product_id.1,product_name,product_price,total_amount,order_year_month,order_month
0,67,Gordy Matteau,59 Northland Circle,Wilmington,DE,19805.0,1001,2020-01-11,67,1001,106,3.0,106,Great value Juice,$1.99,$5.97,2020-01,January
1,201,Cort Paskerful,769 Buell Drive,Huntsville,AL,35810.0,1002,2019-11-22,201,1002,104,2.0,104,Desk,$85.98,$171.96,2019-11,November
2,45,Willetta Lumley,41 Ridgeway Alley,Irvine,CA,92612.0,1003,2019-09-17,45,1003,107,5.0,107,Dove deodorant,$3.99,$19.95,2019-09,September
3,106,Alison Batterton,489 Erie Hill,Richmond,VA,23293.0,1004,2019-12-09,106,1004,106,8.0,106,Great value Juice,$1.99,$15.92,2019-12,December
4,357,Martelle Paxeford,451 Manitowish Place,Richmond,VA,23293.0,1005,2019-06-08,357,1005,107,7.0,107,Dove deodorant,$3.99,$27.93,2019-06,June
5,351,Glenn Reville,3 Springview Drive,Dallas,TX,75277.0,1006,2020-02-18,351,1006,106,4.0,106,Great value Juice,$1.99,$7.96,2020-02,February
6,233,Cosette Berkowitz,7619 Susan Lane,South Bend,IN,46634.0,1007,2019-10-25,233,1007,105,2.0,105,Office Chair,$55.99,$111.98,2019-10,October
7,121,Nisse Cunniam,0 Dennis Pass,Lincoln,NE,68531.0,1008,2019-11-10,121,1008,101,1.0,101,Dell E5470 Laptop,$485.00,$485.00,2019-11,November
8,232,Yank Eayrs,8176 Lawn Drive,Washington,DC,20575.0,1009,2019-09-27,232,1009,110,1.0,110,Apple iPad Mini,$329.99,$329.99,2019-09,September
9,113,Jamesy Brothwood,0693 Evergreen Pass,Scranton,PA,18514.0,1010,2019-09-02,113,1010,108,1.0,108,Printer,$59.99,$59.99,2019-09,September


In [8]:
#look at the datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9569 entries, 0 to 9568
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       9569 non-null   int64  
 1   customer_name     9569 non-null   object 
 2   address_line_1    9448 non-null   object 
 3   city              9516 non-null   object 
 4   state_province    9569 non-null   object 
 5   postal_code       9416 non-null   float64
 6   order_id          9569 non-null   int64  
 7   order_date        9312 non-null   object 
 8   customer_id.1     9569 non-null   int64  
 9   order_id.1        9569 non-null   int64  
 10  product_id        9569 non-null   int64  
 11  quantity          9513 non-null   float64
 12  product_id.1      9569 non-null   int64  
 13  product_name      9569 non-null   object 
 14  product_price     9569 non-null   object 
 15  total_amount      9513 non-null   object 
 16  order_year_month  9312 non-null   object 


In [9]:
#describe the data before cleaning it
df.describe()

,customer_id,postal_code,order_id,customer_id.1,order_id.1,product_id,quantity,product_id.1
count,9569.000000,9416.000000,9569.000000,9569.000000,9569.000000,9569.000000,9513.000000,9569.000000
mean,282.910962,52225.007116,5107.539450,282.910962,5107.539450,106.736650,3.453380,106.736650
std,199.763947,29619.544418,2043.449403,199.763947,2043.449403,3.694805,4.393894,3.694805
min,1.000000,1105.000000,1001.000000,1.000000,1001.000000,101.000000,1.000000,101.000000
25%,127.000000,27409.000000,3393.000000,127.000000,3393.000000,104.000000,1.000000,104.000000
50%,253.000000,48295.000000,5577.000000,253.000000,5577.000000,107.000000,1.000000,107.000000
75%,376.000000,79923.000000,6829.000000,376.000000,6829.000000,110.000000,4.000000,110.000000
max,800.000000,99812.000000,8147.000000,800.000000,8147.000000,114.000000,20.000000,114.000000


## Clean up the data
Did you notice anything unusual about the "house" amounts? 

How about the handset price and income?

Clean up the data in a  new datafram named "df_clean"


In [23]:
#delete any rows with missing values in the clean dataframe
df_clean = df
df_clean = df_clean.dropna(subset=['postal_code','order_date','quantity']) 

## clean total amount column from $ and , signs to be able to sum it when grouping
df_clean['total_amount'] = df_clean['total_amount'].str.replace("$", "", regex=True)
df_clean['total_amount'] = df_clean['total_amount'].str.replace(",", "", regex=True)
df_clean['total_amount'] = df_clean['total_amount'].astype(float)

df_clean['product_price'] = df_clean['product_price'].str.replace("$", "", regex=True)
df_clean['product_price'] = df_clean['product_price'].str.replace(",", "", regex=True)
df_clean['product_price'] = df_clean['product_price'].astype(float)

df_clean['order_year_month'] = pd.to_datetime(df_clean['order_year_month'])
df_clean['order_year_month'] = df_clean['order_year_month'].map(dt.datetime.toordinal)

#delete any rows with missing values in the clean dataframe
df_clean = df_clean.dropna() 

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8981 entries, 0 to 9568
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       8981 non-null   int64  
 1   customer_name     8981 non-null   object 
 2   address_line_1    8981 non-null   object 
 3   city              8981 non-null   object 
 4   state_province    8981 non-null   object 
 5   postal_code       8981 non-null   float64
 6   order_id          8981 non-null   int64  
 7   order_date        8981 non-null   object 
 8   customer_id.1     8981 non-null   int64  
 9   order_id.1        8981 non-null   int64  
 10  product_id        8981 non-null   int64  
 11  quantity          8981 non-null   float64
 12  product_id.1      8981 non-null   int64  
 13  product_name      8981 non-null   object 
 14  product_price     8981 non-null   float64
 15  total_amount      8981 non-null   float64
 16  order_year_month  8981 non-null   int64  


<ipython-input-23-4f841a1c5fe7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['total_amount'] = df_clean['total_amount'].str.replace("$", "", regex=True)
<ipython-input-23-4f841a1c5fe7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['total_amount'] = df_clean['total_amount'].str.replace(",", "", regex=True)
<ipython-input-23-4f841a1c5fe7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

# Standardize attributes

- Convert "college" to 0,1
- Convert reported_satisfaction to 1,2,3
- Convert reported_usage_level to 1,2,3
- Convert considering_change_of_plan to 0,1
- Convert leave to 0,1

I'm going to leave the "leave" variable in place, just in case I need the original value

In [15]:
df_clean.columns

Index(['customer_id', 'customer_name', 'address_line_1', 'city',
       'state_province', 'postal_code', 'order_id', 'order_date',
       'customer_id.1', 'order_id.1', 'product_id', 'quantity', 'product_id.1',
       'product_name', 'product_price', 'total_amount', 'order_year_month',
       'order_month'],
      dtype='object')

In [16]:
df_clean['order_year_month'].unique()

array(['2020-01', '2019-11', '2019-09', '2019-12', '2019-06', '2020-02',
       '2019-10', '2019-05', '2019-07', '2019-08', '2019-04', '2019-03',
       '2019-02', '2020-04', '2020-05', '2020-03', '2021-01', '2020-06',
       '2020-12', '2020-11', '2020-10', '2020-09', '2020-07', '2020-08',
       '2021-05', '2021-08', '2021-03', '2021-06', '2021-07', '2021-04',
       '2021-02'], dtype=object)

In [32]:
x = df_clean['product_price']
y = df_clean['total_amount']

# Note the difference in argument order
model = sm.OLS(y, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           total_amount   R-squared (uncentered):                   0.912
Model:                            OLS   Adj. R-squared (uncentered):              0.912
Method:                 Least Squares   F-statistic:                          9.286e+04
Date:                Wed, 12 Apr 2023   Prob (F-statistic):                        0.00
Time:                        22:54:03   Log-Likelihood:                         -57823.
No. Observations:                8981   AIC:                                  1.156e+05
Df Residuals:                    8980   BIC:                                  1.157e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
product_price     1.0871      0.004    304.732      0.000       1.080       1.094
==============================================================================
Omnibus:                     6597.243   Durbin-Watson:                   1.847
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            93409.599
Skew:                           3.531   Prob(JB):                         0.00
Kurtosis:                      17.133   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
x = df_clean[['product_price','quantity']]
y = df_clean['total_amount']
lm = linear_model.LinearRegression()
model = lm.fit(x,y)

predictions = lm.predict(x)
print(predictions)

[ 26.73035715 105.01529456  54.64121046 ... 356.97128806  28.62043367
  67.23078833]


In [39]:
lm.score(x,y)

0.8650549945430357

In [40]:
lm.intercept_

-14.03976129942481